RESOURCES:                                                                  \
https://python-overpy.readthedocs.io/en/latest/index.html                   \
https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_API_by_Example    \
https://www.openstreetmap.org/relation/7486148                              \
https://overpass-turbo.eu/    

In [7]:
import overpy
import pandas as pd

In [8]:
api = overpy.Overpass()

In [34]:
# Declare tags of interest

supermarket_tag = "'shop'='supermarket'"
convenience_tag = "'shop'='convenience'"
variety_store_tag = "'shop'='variety_store'"
general_store_tag = "'shop'='general'"

library_tag = "'amenity'='library'"
cafe_tag = "'amenity'='cafe'"
pub_tag = "'amenity'='pub'"
bar_tag = "'amenity'='bar'"
fast_food_tag = "'amenity'='fast_food'"
place_of_worship_tag = "'amenity'='place_of_worship'"
marketplace_tag = "'amenity'='marketplace'"  #e.g., farmer's market
bank_tag = "'amenity'='bank'"
pharmacy_tag = "'amenity'='pharmacy'"
ice_cream_tag = "'amenity'='ice_cream'"

In [10]:
# Construct area ID

relation_id = 7486148  #Guelph
OSM_base = 3600000000
area_id = str(OSM_base + relation_id)

In [48]:
# A query function to get the Overpass API data
def osm_query(area_id,tag,element_type):

    result = api.query("""
        [out:json];
        area("""+area_id+""");
        ("""+element_type+"""["""+tag+"""](area);
        );
        out center;
        """)

    print("Number of " + element_type + "s found in area of interest:", len(result.nodes))

    return result

In [49]:
# Run query 
query_result = osm_query(area_id, "shop", "node")

Number of Nodes found in 3607486148: 480


In [54]:
data_dict = []

for node in query_result.nodes:
    data_dict.append({
                'ID': node.id,
                'Name': node.tags.get("name", ""), 
                'Value': node.tags.get("shop", ""), 
                'Latitude': node.lat,
                'Longitude': node.lon
             })

result_db = pd.DataFrame(data_dict)

In [55]:
# Examine results 
result_db.head(25)

,ID,Name,Value,Latitude,Longitude
0,674506769,,convenience,43.5671518,-80.2222304
1,1357939643,Food Basics,supermarket,43.4999439,-80.1917944
2,1357939651,No Frills,supermarket,43.5249669,-80.2222102
3,1358009564,LCBO,alcohol,43.5140995,-80.2394903
4,1528731304,West End Bakery,bakery,43.5467183,-80.2496206
5,1583405191,Food Basics,supermarket,43.5434579,-80.2867841
6,1732491792,,hairdresser,43.5087841,-80.2304978
7,1732492157,,butcher,43.5084826,-80.2308277
8,1783614481,,convenience,43.5776437,-80.2608631
9,2033405315,Zehrs,supermarket,43.5229125,-80.2895504
